In [ ]:
start_date= datetime.strptime(f'{ano}-01-01', '%Y-%m-%d')
end_date = datetime.strptime(f'{ano}-12-31', '%Y-%m-%d')

df_acerto = pd.DataFrame(matrix_acerto, columns=dfFinal.columns)
df_acerto['Data da Operação'] = pd.to_datetime(df_acerto['Data da Operação'], format='%d/%m/%Y')
mask = (df_acerto['Data da Operação'] >= start_date) & (df_acerto['Data da Operação'] <= end_date)
df_acerto = df_acerto.loc[mask]

df = pd.concat([dfFinal, df_acerto], ignore_index=True)

In [ ]:
dfFinal = df.sort_values(['Data da Operação', 'Tipo Operação'], ascending=[True, False])

In [ ]:
# Ajustando caracter de decimal e milhar em "Resultado (R$)", e conversão para float
dfFinal['Resultado (R$)'] = [x.replace('.','') for x in dfFinal['Resultado (R$)']]
dfFinal['Resultado (R$)'] = [x.replace(',','.') for x in dfFinal['Resultado (R$)']]
dfFinal['Resultado (R$)'] = dfFinal['Resultado (R$)'].astype(float)

In [ ]:
# Removendo sinal de % para cálculos, e conversão para float
dfFinal['%'] = [x.replace(' %','') for x in dfFinal['%']]
dfFinal['%'] = [x.replace(',','.') for x in dfFinal['%']]
dfFinal['%'] = dfFinal['%'].astype(float)

In [ ]:
dfFinal = dfFinal.drop(['Característica', 'Corretora'], axis=1)

In [ ]:
dfFinal['Qtde'] = dfFinal['Qtde'].astype(str)
dfFinal['Qtde'] = [x.replace('.','') for x in dfFinal['Qtde']]
dfFinal['Qtde'] = dfFinal['Qtde'].astype(int)

In [ ]:
# Excluir linha de acerto da planilha caso o valor já esteja lançado.
if dfFinal.iloc[-1]['Resultado (R$)'] == dfFinal.iloc[-2]['Resultado (R$)']:
    dfFinal=dfFinal.drop(dfFinal.index[-1])

In [ ]:
dfFinal['Preço (R$)'] = dfFinal['Preço (R$)'].astype(float)

In [ ]:
# Calcula vr_compra
vr_compra_bruto = dfFinal['Qtde'] * dfFinal['Preço (R$)']

In [ ]:
emolumentos = 325 / 1000000
corretagem = 49 / 10
vr_compra = vr_compra_bruto + corretagem + (vr_compra_bruto * emolumentos)

In [ ]:
dfFinal['Valor Compra'] = vr_compra

In [ ]:
# Cria Dataframe com apenas Porcentagem e Lucro/Perda (R$)
df_sum = dfFinal[['%', 'Resultado (R$)']].copy()

In [ ]:
df_sum = df_sum.rename(columns = {"%" : "Porcentagem"})

In [ ]:
# Corrigir erro na GOLL4, dia 27/05/2025
result = dfFinal[dfFinal["Resultado (R$)"] == -24346.77]
if not result.empty:
    result_index = result.index[0]
    dfFinal.at[result_index, "Resultado (R$)"] = 2769.48
    dfFinal.at[result_index, "%"] = 4.10

In [ ]:
# Corrigir valores de GOLL54, que tem fator 1000
dfFinal["Valor Compra"] = np.where(dfFinal["Ativo"] == "GOLL54", dfFinal["Valor Compra"] / 1000, dfFinal["Valor Compra"])